<a href="https://colab.research.google.com/github/Sammywarah/dsc-website-ab-testing-lab/blob/master/Website_A_B_Testing_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups?
            * If so, how do you plan to account for this in your experimental design?

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/homepage_actions.csv')

# Display the first few rows of the dataset to get an overview
print(df.head())

# Check the number of viewers who clicked
viewers_clicked = df[df['action'] == 'click']['id'].nunique()
print("Number of viewers who clicked:", viewers_clicked)

# Check for any anomalies - viewers who clicked without viewing
click_without_view = df[df['action'] == 'click']['id'].isin(df[df['action'] == 'view']['id']).sum()
print("Number of clicks without view:", click_without_view)

# Check for overlap between control and experiment groups
control_group = df[df['group'] == 'control']['id'].unique()
experiment_group = df[df['group'] == 'experiment']['id'].unique()
overlap = len(set(control_group).intersection(experiment_group))
print("Number of users in both control and experiment groups:", overlap)


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
Number of viewers who clicked: 1860
Number of clicks without view: 1860
Number of users in both control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
#Your code here
import statsmodels.api as sm

# Calculate the number of viewers and clickers in each group
control_viewers = df[(df['group'] == 'control') & (df['action'] == 'view')]['id'].nunique()
control_clickers = df[(df['group'] == 'control') & (df['action'] == 'click')]['id'].nunique()

experiment_viewers = df[(df['group'] == 'experiment') & (df['action'] == 'view')]['id'].nunique()
experiment_clickers = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique()

# Calculate the click-through rates (CTR) for each group
control_ctr = control_clickers / control_viewers
experiment_ctr = experiment_clickers / experiment_viewers

# Perform the z-test
z_score, p_value = sm.stats.proportions_ztest([control_clickers, experiment_clickers], [control_viewers, experiment_viewers])

# Print the results
print("Control CTR:", control_ctr)
print("Experiment CTR:", experiment_ctr)
print("Z-score:", z_score)
print("P-value:", p_value)


Control CTR: 0.2797118847539016
Experiment CTR: 0.3097463284379172
Z-score: -2.618563885349469
P-value: 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not.

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate.
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.

In [4]:
#Your code here
expected_experiment_clicks = experiment_viewers * control_ctr
print("Expected number of clicks for the experiment group:", expected_experiment_clicks)

Expected number of clicks for the experiment group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [6]:
#Your code here
import math
std_dev_experiment_clicks = math.sqrt(experiment_viewers * control_ctr * (1 - control_ctr))

# Calculate the z-score
z_score = (experiment_clickers - expected_experiment_clicks) / std_dev_experiment_clicks
print("Number of standard deviations:", z_score)

Number of standard deviations: 3.6625360854823588


### Step 3:
Finally, calculate a p-value using the normal distribution based on this z-score.

In [7]:
#Your code here
from scipy.stats import norm

# Step 3: Calculate the p-value using the normal distribution
p_value = 1 - norm.cdf(z_score)
print("P-value based on the z-score:", p_value)


P-value based on the z-score: 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: While the values are not exactly the same, they are both relatively low (less than the conventional significance level of 0.05), indicating statistical significance. Additionally, both p-values suggest evidence against the null hypothesis, which is consistent with each other.

Therefore, despite the slight discrepancy in the exact values, the results from both approaches align in indicating that the experimental homepage was more effective than the control group.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.